In [1]:
from tensorflow_model import train, load_image, predict
import Contrastive_Explanation_Method_master.Utils as utils
from Contrastive_Explanation_Method_master.aen_CEM import AEADEN
import random
import tensorflow as tf
import numpy as np
from dataset_handler import DatasetHandler
from tqdm import tqdm
import os

Using TensorFlow backend.


# 1. Load pretrained model

In [2]:
model = tf.keras.applications.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# 2. Predict with pretrained model

In [3]:
# auffällig: Prediction ist deutlich unsicher als über PyTorch oder über Tensorflow 2.0 Model
# Wurden hier vielleicht einfach noch nicht die Wahrscheinlichkeiten normalisiert?
img = load_image("dog.jpg")
preds = predict(model, img)

(224, 224, 3)
(1, 224, 224, 3)
Predicted: [('n02111889', 'Samoyed', 0.15781638), ('n02120079', 'Arctic_fox', 0.09153663), ('n02098286', 'West_Highland_white_terrier', 0.03576579)]


# 3. CEM

In [4]:
### Probleme:

# VGG laden -> tf 2  --> vielleicht musste nur zusätzlich zu tf auch keras in version 1.15 installiert werden, damit klappt
# CEM Implementierung -> tf 1

In [5]:
# Model class for the use in __init__ of AEDM() inspiried by class MNISTModel
class Model():
    
    def __init__(self, model, image_size, num_channels, num_labels):
        self.model = model
        self.image_size = image_size
        self.num_channels = num_channels
        self.num_labels = num_labels
        
    def predict(self, data):
        return self.model(data)

In [6]:
# Data class for the use of utils.generate_data
class Data():
    def __init__(self, test_data, test_labels):
        self.test_data = test_data
        self.test_labels = test_labels

In [7]:
# define data
test_data = np.expand_dims(img, axis=0)
target_label = preds.argmax()
test_labels = np.expand_dims(np.eye(1000)[target_label], axis=0)
image_id = 0


data = Data(test_data, test_labels)

In [8]:
# define paramters (was ist arg_b??)
arg_max_iter = 500
arg_b = 5 # binary_search_steps
arg_init_const = 0.1
arg_mode = "PP"
arg_kappa = 10
arg_beta = 0.1
arg_gamma = 0

In [10]:
with tf.Session() as sess:
    # load model after creating Session
    model_vgg = tf.keras.applications.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
    )
    
    model = Model(model=model_vgg, image_size=224, num_channels=3, num_labels=1000)

    random.seed(121)
    np.random.seed(1211)
    orig_prob, orig_class, orig_prob_str = utils.model_prediction(model, np.expand_dims(img, axis=0))
    target_label = orig_class
    print("Image:{}, infer label:{}".format(image_id, target_label))
    
    
    orig_img, target = utils.generate_data(data, image_id, target_label)
    
    
    
    

    attack = AEADEN(sess, model, mode = arg_mode, AE = lambda x: x, batch_size=1, kappa=arg_kappa, init_learning_rate=1e-2,
        binary_search_steps=arg_b, max_iterations=arg_max_iter, initial_const=arg_init_const, beta=arg_beta, gamma=arg_gamma)

    adv_img = attack.attack(orig_img, target)

    adv_prob, adv_class, adv_prob_str = utils.model_prediction(model, adv_img)
    delta_prob, delta_class, delta_prob_str = utils.model_prediction(model, orig_img-adv_img)

    INFO = "[INFO]id:{}, kappa:{}, Orig class:{}, Adv class:{}, Delta class: {}, Orig prob:{}, Adv prob:{}, Delta prob:{}".format(image_id, arg_kappa, orig_class, adv_class, delta_class, orig_prob_str, adv_prob_str, delta_prob_str)
    print(INFO)

    suffix = "id{}_kappa{}_Orig{}_Adv{}_Delta{}".format(image_id, arg_kappa, orig_class, adv_class, delta_class)
    arg_save_dir = "{}_ID{}_Gamma_{}".format(arg_mode, image_id, arg_gamma)
    os.system("mkdir -p Results/{}".format(arg_save_dir))
    utils.save_img(orig_img, "Results/{}/Orig_original{}.png".format(arg_save_dir, orig_class))
    utils.save_img(adv_img, "Results/{}/Adv_{}.png".format(arg_save_dir, suffix))
    utils.save_img(np.absolute(orig_img-adv_img)-0.5, "Results/{}/Delta_{}.png".format(arg_save_dir, suffix))

    sys.stdout.flush()

Image:0, infer label:258
<class 'tensorflow.python.framework.ops.Tensor'>


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

!!!!! (1, 224, 224, 3)

iter:0 const:[0.1]
Loss_Overall:1.0012, Loss_Attack:1.0012
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:50 const:[0.1]
Loss_Overall:1.0012, Loss_Attack:1.0012
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:100 const:[0.1]
Loss_Overall:1.0012, Loss_Attack:1.0012
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:150 const:[0.1]
Loss_Overall:1.0012, Loss_Attack:1.0012
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:200 const:[0.1]
Loss_Overall:1.0012, Loss_Attack:1.0012
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000

 20%|████████████████▌                                                                  | 1/5 [06:51<27:26, 411.52s/it]

!!!!! (1, 224, 224, 3)

iter:0 const:[1.]
Loss_Overall:10.0124, Loss_Attack:10.0124
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:50 const:[1.]
Loss_Overall:10.0124, Loss_Attack:10.0124
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:100 const:[1.]
Loss_Overall:10.0124, Loss_Attack:10.0124
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:150 const:[1.]
Loss_Overall:10.0124, Loss_Attack:10.0124
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:200 const:[1.]
Loss_Overall:10.0124, Loss_Attack:10.0124
Loss_L2Dist:0.0000, Loss_L1Dist:0

...

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [25:38<06:10, 370.82s/it]

!!!!! (1, 224, 224, 3)

iter:0 const:[1000.]
Loss_Overall:10012.3496, Loss_Attack:10012.3496
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:50 const:[1000.]
Loss_Overall:10012.3496, Loss_Attack:10012.3496
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:100 const:[1000.]
Loss_Overall:10012.3496, Loss_Attack:10012.3496
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:150 const:[1000.]
Loss_Overall:10012.3496, Loss_Attack:10012.3496
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:0.0040, max_nontarget_lab_score:0.0164



















































iter:200 const:[1000.]
Loss_Overall:10012.3496, Loss_Attack

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [31:54<00:00, 382.82s/it]


[INFO]id:0, kappa:10, Orig class:258, Adv class:669, Delta class: 258, Orig prob:[[2.41664166e-04 8.32844176e-04 8.21235066e-04 8.18387663e-04  2.32763519e-03 1.01569085e-03 9.22699738e-03 2.49907695e-04  4.79905138e-04 3.57810903e-04 4.25206643e-04 2.48361932e-04  3.47115565e-04 1.32405257e-03 1.97975518e-04 1.79150200e-04  5.56292129e-04 4.48793900e-04 5.81041211e-04 9.28692927e-04  1.16436544e-03 2.29528197e-03 1.68804557e-03 1.28950877e-03  1.40219883e-04 3.82209473e-05 3.48781672e-04 1.93879154e-04  3.33993703e-05 7.66462600e-03 7.36966613e-05 7.77517562e-05  4.68839353e-05 1.23205711e-03 1.16048986e-03 1.01562684e-04  3.89757537e-04 4.12723057e-05 6.39816804e-04 2.24607080e-04  2.83942441e-04 1.19958560e-04 1.89036000e-04 1.17047144e-04  1.53889836e-04 7.39896204e-04 3.19593673e-04 1.98773690e-04  5.96177531e-04 4.88025864e-04 1.00583467e-03 1.28391257e-04  2.81287794e-04 3.76875920e-04 2.20023576e-04 1.25388600e-04  2.07740683e-04 4.72076426e-05 3.86626367e-03 1.17516523e-04  4.

NameError: name 'os' is not defined